Formulate the Knapsack Problem as a Mixed Integer Linear Program

Import Libraries:

In [42]:
from scipy.optimize import milp, Bounds, LinearConstraint
import pandas as pd
import numpy as np

Let's imagine our knapsack is a diaper bag. Let's give some values and weights to some items we'd like to pack in the diaper bag for our afternoon of errands:

In [43]:
df = pd.DataFrame(columns=['weight','value','minimum','maximum'])
df.loc['bottle'] = [10,30,1,np.inf]
df.loc['burp_cloth'] = [5,5,0,np.inf]
df.loc['coffee'] = [10,15,0,np.inf]
df.loc['diaper'] = [3,20,1,6]
df.loc['extra_clothes'] = [10,5,0,np.inf]
df.loc['keys'] = [3,50,1,1]
df.loc['pacifier'] = [0.5,10,0,3]
df.loc['teddy_bear'] = [20,5,0,np.inf]
df.loc['wallet'] = [5,50,1,1]
df.loc['water_bottle'] = [20,8,0,np.inf]
df.loc['wipes'] = [10,20,1,1]

print(df)

               weight  value  minimum  maximum
bottle           10.0   30.0      1.0      inf
burp_cloth        5.0    5.0      0.0      inf
coffee           10.0   15.0      0.0      inf
diaper            3.0   20.0      1.0      6.0
extra_clothes    10.0    5.0      0.0      inf
keys              3.0   50.0      1.0      1.0
pacifier          0.5   10.0      0.0      3.0
teddy_bear       20.0    5.0      0.0      inf
wallet            5.0   50.0      1.0      1.0
water_bottle     20.0    8.0      0.0      inf
wipes            10.0   20.0      1.0      1.0


scipy.optimize.milp treats the objective function as something that is to be minimized, so the objective function must be written accordingly. We want to maximize the value of the objects in the knapsack, but to fit within scipy's framework, we must write this as "minimize the negative total value in the knapsack".

In [44]:
def get_objective(df):
    negative = -df['value']
    return negative.tolist()

In [45]:
def get_constraints(df,ub):
    return LinearConstraint(A=df['weight'].tolist(),ub=ub)

In [46]:
def get_bounds(df):
    minimums = df['minimum'].tolist()
    maximums = df['maximum'].tolist()
    return Bounds(lb=minimums,ub=maximums)

In [47]:
result = milp(c=get_objective(df),integrality=1,bounds=get_bounds(df),constraints=get_constraints(df,100))

In [48]:
with np.printoptions(formatter={'float': '{:.0f}'.format}):
    print(f"Total Value: {-result.fun:.0f}")
    df['quantities'] = result.x
    print(df)

Total Value: 450
               weight  value  minimum  maximum  quantities
bottle           10.0   30.0      1.0      inf         6.0
burp_cloth        5.0    5.0      0.0      inf         0.0
coffee           10.0   15.0      0.0      inf         0.0
diaper            3.0   20.0      1.0      6.0         6.0
extra_clothes    10.0    5.0      0.0      inf         0.0
keys              3.0   50.0      1.0      1.0         1.0
pacifier          0.5   10.0      0.0      3.0         3.0
teddy_bear       20.0    5.0      0.0      inf         0.0
wallet            5.0   50.0      1.0      1.0         1.0
water_bottle     20.0    8.0      0.0      inf         0.0
wipes            10.0   20.0      1.0      1.0         1.0
